In [7]:
import pandas as pd

file_path = r'C:\Users\Erione Technologies\ERIVAL TELECOM COM E REPRES LT\IT Operations - ERIONE - Documentos\Interno\ControleColaboradores\Base .Csv\tinnova-usuarios.csv'

df = pd.read_csv(file_path, sep=',')

output_file_name = 'Colaboradores_Tinnova_Modelo.xlsx'

output_directory = r'C:\Users\Erione Technologies\ERIVAL TELECOM COM E REPRES LT\IT Operations - ERIONE - Documentos\Interno\ControleColaboradores\Colaboradores_Modelo'

output_path = f'{output_directory}\\{output_file_name}'

df.to_excel(output_path, index=False)

ParserError: Error tokenizing data. C error: Expected 28 fields in line 170, saw 29


In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from pathlib import Path
from datetime import datetime
import logging

# Configurações
INPUT_FILE_PATH = Path(r'C:\Users\Erione Technologies\ERIVAL TELECOM COM E REPRES LT\IT Operations - ERIONE - Documentos\Interno\ControleColaboradores\Colaboradores_Modelo\Colaboradores_Tinnova_Modelo.xlsx')
OUTPUT_DIRECTORY = Path(r'C:\Users\Erione Technologies\ERIVAL TELECOM COM E REPRES LT\IT Operations - ERIONE - Documentos\Interno\ControleColaboradores\Controle_Mensais\Tinnova')

COLUNAS_PARA_REMOVER = [
    'DirSyncEnabled', 'ID do Objeto', 'Nome', 'Sobrenome', 'Data/hora de criação',
    'Carimbo de data/hora da exclusão reversível.', 'Título', 'Departamento',
    'Localização de dados preferida', 'Cidade', 'CountryOrRegion', 'Office',
    'StateOrProvince', 'Local de uso', 'Última execução do dirsync',
    'A senha nunca expira', 'Carimbo de data/hora da última alteração de senha',
    'Telefone Celular', 'Número de telefone', 'CEP', 'Idioma preferencial',
    'Endereço', 'Fax', 'Endereços de proxy'
]

RENOMEAR_COLUNAS = {
    'Nome para exibição': 'Nome do Colaborador',
    'Nome UPN': 'Email',
    'Bloquear credencial': 'Conta desativada',
    'Licenças': 'licencas',
}

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def carregar_dados(file_path: Path) -> pd.DataFrame:
    """Carrega e prepara os dados iniciais do arquivo Excel."""
    try:
        logger.info(f"Carregando dados de {file_path}")
        df = pd.read_excel(file_path, engine='openpyxl')
        
        # Verificar colunas existentes
        cols_existentes = [col for col in RENOMEAR_COLUNAS if col in df.columns]
        df = df.rename(columns={k: v for k, v in RENOMEAR_COLUNAS.items() if k in cols_existentes})
        
        # Remover colunas
        cols_para_remover = [col for col in COLUNAS_PARA_REMOVER if col in df.columns]
        df = df.drop(columns=cols_para_remover, errors='ignore')
        
        # Filtrar emails (mantendo apenas os que não contêm o padrão)
        if 'Email' in df.columns:
            mask = ~df['Email'].astype(str).str.contains('#EXT#@tarumainfo.onmicrosoft.com', na=False)
            df = df[mask].copy()
        
        return df
    
    except Exception as e:
        logger.error(f"Erro ao carregar dados: {str(e)}")
        raise

def processar_licencas(df: pd.DataFrame) -> pd.DataFrame:
    """Processa as colunas de licenças usando MultiLabelBinarizer."""
    if 'licencas' not in df.columns:
        logger.warning("Coluna 'licencas' não encontrada - pulando processamento")
        return df
    
    try:
        logger.info("Processando coluna de licenças")
        
        # Converter licenças para lista
        df['licencas'] = df['licencas'].apply(
            lambda x: x.split('+') if pd.notna(x) and isinstance(x, str) else []
        )
        
        # Aplicar MultiLabelBinarizer
        mlb = MultiLabelBinarizer()
        binary_licenses = mlb.fit_transform(df['licencas'])
        binary_licenses_df = pd.DataFrame(
            binary_licenses, 
            columns=mlb.classes_,
            index=df.index  # Manter o mesmo índice para evitar problemas
        ).replace({1: "VERDADEIRO", 0: "FALSO"})
        
        # Juntar com o DataFrame original
        df = pd.concat([df, binary_licenses_df], axis=1)
        
        # Reorganizar colunas
        colunas = [col for col in df.columns if col != 'licencas']
        
        if 'Não licenciado' in colunas:
            colunas.remove('Não licenciado')
            colunas.insert(3, 'Não licenciado')
        
        colunas.insert(11, 'licencas')
        
        return df[colunas].drop(columns=['licencas'], errors='ignore')
    
    except Exception as e:
        logger.error(f"Erro ao processar licenças: {str(e)}")
        raise

def salvar_resultado(df: pd.DataFrame, output_dir: Path) -> Path:
    """Salva o dataframe processado em um arquivo Excel."""
    try:
        output_dir.mkdir(parents=True, exist_ok=True)
        current_date = datetime.now().strftime('%Y-%m-%d')
        output_file_name = f'Colaboradores_Tinnova_Modelo_{current_date}.xlsx'
        output_file_path = output_dir / output_file_name
        
        # Configurar engine para openpyxl e garantir que não pule linhas
        with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
            df.to_excel(writer, index=False)
        
        logger.info(f"Arquivo salvo com sucesso: {output_file_path}")
        return output_file_path
    
    except Exception as e:
        logger.error(f"Erro ao salvar resultado: {str(e)}")
        raise

def main():
    try:
        logger.info("Iniciando processamento do arquivo de colaboradores da Tinnova")
        
        # Carregar dados
        df = carregar_dados(INPUT_FILE_PATH)
        
        # Processar licenças
        df = processar_licencas(df)
        
        # Remover linhas completamente vazias que possam ter sido criadas
        df = df.dropna(how='all')
        
        # Salvar resultado
        salvar_resultado(df, OUTPUT_DIRECTORY)
        
        logger.info("Processamento concluído com sucesso")
        
    except Exception as e:
        logger.error(f"Erro durante o processamento: {str(e)}", exc_info=True)
        raise

if __name__ == "__main__":
    main()

INFO:__main__:Iniciando processamento do arquivo de colaboradores da Tinnova
INFO:__main__:Carregando dados de C:\Users\Erione Technologies\ERIVAL TELECOM COM E REPRES LT\IT Operations - ERIONE - Documentos\Interno\ControleColaboradores\Colaboradores_Modelo\Colaboradores_Tinnova_Modelo.xlsx
INFO:__main__:Arquivo salvo com sucesso: C:\Users\Erione Technologies\ERIVAL TELECOM COM E REPRES LT\IT Operations - ERIONE - Documentos\Interno\ControleColaboradores\Controle_Mensais\Tinnova\Colaboradores_Tinnova_Modelo_2025-05-06.xlsx
INFO:__main__:Processamento concluído com sucesso
